In [1]:
import numpy as np # linear algebra
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
#Dependencies
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
#CNN
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Dense, Flatten, GlobalMaxPooling2D
from keras.callbacks import CSVLogger
from livelossplot.keras import PlotLossesCallback
import efficientnet.keras as efn

from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [3]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [4]:
TRAINING_LOGS_FILE = "training_logs.csv"
MODEL_SUMMARY_FILE = "model_summary.txt"

In [5]:
datasetFolderName='datasets4'
MODEL_FILENAME="model_cv.h5"
sourceFiles=[]
classLabels=['Background', 'Ferny', 'Strappy','Rounded']
def transferBetweenFolders(source, dest, splitRate):   
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', 
                               splitRate)


In [6]:
# First, check if test folder is empty or not, if not transfer all existing files to train
#transferAllClassBetweenFolders('test', 'train', 1.0)
# Now, split some part of train data into the test folders.
#transferAllClassBetweenFolders('train', 'test', 0.20)


In [7]:
X=[]
Y=[]

In [8]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        if(folderName==classLabels[0]):
            Y.append(0)
        elif(folderName==classLabels[1]):
            Y.append(1)
        elif(folderName==classLabels[2]):
            Y.append(2)
        else:
            Y.append(3)

In [9]:
# Organize file names and class labels in X and Y variables
prepareNameWithLabels(classLabels[0])
prepareNameWithLabels(classLabels[1])
prepareNameWithLabels(classLabels[2]) 
prepareNameWithLabels(classLabels[3]) 


In [10]:
X=np.asarray(X)
Y=np.asarray(Y)

In [11]:
IMAGE_SIZE = 456
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
#EPOCHS = 10
BATCH_SIZE = 32
TEST_SIZE = 30

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

In [12]:
# learning rate

#activationFunction='relu'
def getModel():
    
    model = Sequential()
    efficient_net = efn.EfficientNetB5(weights='imagenet', include_top=False, input_shape=input_shape)
    #efficient_net.trainable = False
    for index, layer in enumerate(efficient_net.layers):
        if index < 761:
            layer.trainable = False

  
    model.add(efficient_net)
    #model.add(GlobalMaxPooling2D())
    model.add(Dense(1024, activation='relu'))
    model.add(Flatten())
    # if dropout_rate > 0:
    #     model.add(layers.Dropout(dropout_rate, name="dropout_out"))
    # model.add(layers.Dense(256, activation='relu', name="fc1"))
    model.add(Dense(4, activation='softmax')) #, name="output"
    model.compile(loss='categorical_crossentropy',
            optimizer=RMSprop(lr=0.0001),
            metrics=['accuracy'])

    with open(MODEL_SUMMARY_FILE,"w") as fh:
        model.summary(print_fn=lambda line: fh.write(line + "\n"))
       
    return model

In [13]:
def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score

In [14]:
# input image dimensions
img_rows, img_cols =  456, 456


In [15]:
train_path=datasetFolderName+'/train/'
validation_path=datasetFolderName+'/validation/'
test_path=datasetFolderName+'/test/'
model=getModel()

In [16]:
checkpoint = ModelCheckpoint("effee44.h5", monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='accuracy', min_delta=0, patience=5, verbose=1, mode='auto')


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        if(Y_val[eachIndex]==0):
            classLabel=classLabels[0]
        elif(Y_val[eachIndex]==1):
            classLabel=classLabels[1]
        elif(Y_val[eachIndex]==2):
            classLabel=classLabels[2]
        else:
            classLabel=classLabels[3]   
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
    train_datagen = ImageDataGenerator(
                rescale=1./255,
        		zoom_range=0.20,
            	fill_mode="nearest"
                )
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
        
    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=32,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
            validation_path,
            target_size=(img_rows, img_cols),
            batch_size=32,
            class_mode=None,  # only data, no labels
            shuffle=False)   
   
    # fit model
    history=model.fit_generator(train_generator, 
                        epochs=5, callbacks = [checkpoint, early])
    
    predictions = model.predict_generator(validation_generator, verbose=0)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")    
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)


Results for fold 1
Found 44638 images belonging to 4 classes.
Found 8930 images belonging to 4 classes.


c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
1395/1395 [==============================] - 2222s 2s/step - loss: 0.5399 - accuracy: 0.8692

Epoch 00001: accuracy improved from -inf to 0.92121, saving model to effee44.h5
Epoch 2/5
1395/1395 [==============================] - 1788s 1s/step - loss: 0.1425 - accuracy: 0.9541

Epoch 00002: accuracy improved from 0.92121 to 0.95602, saving model to effee44.h5
Epoch 3/5
1395/1395 [==============================] - 1706s 1s/step - loss: 0.1246 - accuracy: 0.9619

Epoch 00003: accuracy improved from 0.95602 to 0.96324, saving model to effee44.h5
Epoch 4/5
1395/1395 [==============================] - 1711s 1s/step - loss: 0.1188 - accuracy: 0.9665

Epoch 00004: accuracy improved from 0.96324 to 0.96689, saving model to effee44.h5
Epoch 5/5
1395/1395 [==============================] - 1692s 1s/step - loss: 0.1138 - accuracy: 0.9690

Epoch 00005: accuracy improved from 0.96689 to 0.96828, saving model to effee44.h5


c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


***Performance on Validation data***
Accuracy  : 0.9764837625979843
Precision : 0.9767588773834806
f1Score : 0.9765085199725324
[[1852    1   67    1]
 [   0 1660   45   18]
 [   9   23 2829   14]
 [   0   21   11 2379]]
Results for fold 2
Found 44638 images belonging to 4 classes.
Found 8930 images belonging to 4 classes.


...

Epoch 1/5
1395/1395 [==============================] - 1715s 1s/step - loss: 0.1159 - accuracy: 0.9738

Epoch 00001: accuracy did not improve from 0.97495
Epoch 2/5
1395/1395 [==============================] - 1703s 1s/step - loss: 0.1081 - accuracy: 0.9754

Epoch 00002: accuracy improved from 0.97495 to 0.97536, saving model to effee44.h5
Epoch 3/5
1395/1395 [==============================] - 1697s 1s/step - loss: 0.1027 - accuracy: 0.9767

Epoch 00003: accuracy improved from 0.97536 to 0.97675, saving model to effee44.h5
Epoch 4/5
1395/1395 [==============================] - 1704s 1s/step - loss: 0.1035 - accuracy: 0.9776

Epoch 00004: accuracy improved from 0.97675 to 0.97762, saving model to effee44.h5
Epoch 5/5
1395/1395 [==============================] - 5779s 4s/step - loss: 0.1067 - accuracy: 0.9775

Epoch 00005: accuracy did not improve from 0.97762


c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


***Performance on Validation data***
Accuracy  : 0.9893617021276596
Precision : 0.9894035336838998
f1Score : 0.9893489546094716
[[1920    0    1    0]
 [   2 1682    8   31]
 [  24   13 2833    6]
 [   0    8    2 2400]]
Results for fold 4
Found 44638 images belonging to 4 classes.
Found 8930 images belonging to 4 classes.


c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
1395/1395 [==============================] - 1512s 1s/step - loss: 0.1075 - accuracy: 0.9769

Epoch 00001: accuracy did not improve from 0.97762
Epoch 2/5
 859/1395 [=================>............] - ETA: 9:51 - loss: 0.0983 - accuracy: 0.9783

In [ ]:
# =============TESTING=============
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) 
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)
model.save(MODEL_FILENAME)

In [ ]:
model.save("newnew.pb")

In [55]:
len(X_train)

149

In [56]:
X_train.shape

(149,)

In [57]:
len(Y_train)

149

In [58]:
Y_train.shape


(149,)

In [59]:
len(X_val)

37

In [28]:
X_val.shape

(44,)

In [29]:
len(Y_val)

44

In [30]:
Y_val.shape

(44,)

In [31]:
len(X)

222

In [32]:
len(Y)

222

In [33]:
print(classLabels[2])

Strappy
